In [5]:
# 전처리 단계
import pandas as pd
file_path = 'gangnam_main.csv'
df = pd.read_csv(file_path, encoding='utf-8')

In [6]:
df = df.drop(columns=['번지', '본번', '부번', '동', '도로명', '해제사유발생일', '등기일자', 'addr'])

In [7]:
df['거래금액(백만원)'] = df['거래금액(만원)'].str.replace(',', '').astype(int)/100
df['전용면적'] = df['전용면적(㎡)']

df['계약년월'] = df['계약년월'].astype(str)
df['연도'] = df['계약년월'].str[:4]
df['월'] = df['계약년월'].str[4:6]

df['계약일'] = df['계약일'].astype(str).str.zfill(2)

df['계약일자'] = pd.to_datetime(df['연도'] + '-' + df['월'] + '-' + df['계약일'])

df = df.drop(columns=['연도', '월', '계약년월', '계약일', '거래금액(만원)'])

In [8]:
df[['구', '동']] = df['시군구'].str.extract(r'서울특별시 (\S+구) (\S+동)')

df = df.drop(columns=['시군구'])

In [9]:
interest_rate_df = pd.read_csv('interest_rate.csv')
interest_rate_df['date'] = pd.to_datetime(interest_rate_df['date'])
df['금리'] = None
for index, row in df.iterrows():
    contract_date = row['계약일자']
    previous_rate = interest_rate_df[interest_rate_df['date'] < contract_date]['interest'].iloc[-1] if not interest_rate_df[interest_rate_df['date'] < contract_date].empty else None
    df.at[index, '금리'] = previous_rate


In [10]:
df

,Unnamed: 0,NO,단지명,전용면적(㎡),층,매수자,매도자,건축년도,거래유형,중개사소재지,x,y,subway_dist,거래금액(백만원),전용면적,계약일자,구,동,금리
0,0,1,현대힐스테이트2단지,84.236,16,개인,개인,2008,직거래,-,127.051739,37.514732,0.002086,2195.0,84.236,2024-07-26,강남구,삼성동,3.0
1,1,2,타워팰리스3,163.702,28,개인,개인,2004,중개거래,서울 강남구,127.052530,37.487763,0.004174,4000.0,163.702,2024-07-24,강남구,도곡동,3.0
2,2,3,로덴하우스이스트빌리지,243.360,13,개인,개인,2009,중개거래,"서울 강남구, 서울 용산구",127.046890,37.485672,0.001318,5500.0,243.360,2024-07-23,강남구,도곡동,3.0
3,3,4,강남자곡 힐스테이트,51.960,5,개인,개인,2015,중개거래,서울 강남구,127.088231,37.471853,0.012792,920.0,51.960,2024-07-22,강남구,자곡동,3.0
4,4,5,강남데시앙파크,114.690,8,개인,개인,2011,중개거래,서울 강남구,127.101541,37.461047,0.026344,1590.0,114.690,2024-07-21,강남구,세곡동,3.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
78050,78050,5765,선경3차,76.600,8,-,-,1990,-,-,127.060910,37.494180,0.002549,517.0,76.600,2006-01-02,강남구,대치동,3.75
78051,78051,5766,신동아(22),35.730,9,-,-,1997,-,-,127.023030,37.515025,0.003048,193.0,35.730,2006-01-02,강남구,논현동,3.75
78052,78052,5767,쌍용대치2,84.490,6,-,-,1983,-,-,127.070628,37.499924,0.003387,797.5,84.490,2006-01-02,강남구,대치동,3.75
78053,78053,5768,쌍용대치2,84.490,12,-,-,1983,-,-,127.070628,37.499924,0.003387,800.0,84.490,2006-01-02,강남구,대치동,3.75


In [13]:
features = ['계약일자', '금리', '단지명', '구', '동', '전용면적', '층', '건축년도', 
            '거래유형', '매수자', '매도자', '중개사소재지', 'x', 'y', 'subway_dist', '거래금액(백만원)']
df = df[features]

In [15]:
df.to_csv('Gangnam_features.csv', index=False, encoding='utf-8')